## インストール

In [6]:
pip install pingouin
pip install plotnine
pip install mysql-connector-python
pip install numexpr
pip install python-dotenv
pip install japanize-matplotlib


SyntaxError: invalid syntax (<ipython-input-6-5ad1bb845e3b>, line 2)

In [1]:
from itertools import combinations

import csv
import MySQLdb
import mysql.connector as mydb
import pandas as pd
import pandas.io.sql as psql
import numpy as np
from pingouin import kruskal, mwu, multicomp
from plotnine import *


In [5]:
%load_ext dotenv
%dotenv


In [7]:
host = os.environ.get('HOST')
user = os.environ.get('USER_NAME')
password = os.environ.get('PASSWORD')
database = os.environ.get('DATABASE')
DB_CONFIG = dict(host=host, user=user, password=password, database=database)

def check_db_connection(db_config=DB_CONFIG):
  connection = mydb.connect(**db_config)
  connection.ping(reconnect=True)
  # 接続できているかどうか確認
  print(connection.is_connected())


check_db_connection(DB_CONFIG)


True


In [ ]:
UI_CONDITION = ("question", "perspective", "non")
THEME_CONDITION = ("tpp", "airPollution", "vietnamTrip")

In [11]:

def get_dataframe_from_db(sql):
  """
  db上のデータをデータフレームとして取得
  """
  conn = mydb.connect(**DB_CONFIG)
  df_read = pd.read_sql(sql, conn)
  conn.close()
  return df_read


def convert_csv_to_array(csv_path):
  """
  csvファイルの中身を一次元配列へ変換
  """
  result_arr = []
  with open(csv_path) as f:
      reader = csv.reader(f)
      for row in reader:
        result_arr.append(str(row[0]))
  return result_arr


def convert_singleArray_to_dataframe(array, columnName):
  df = pd.DataFrame(array, columns=[columnName],)
  return df


## 分析対象とする実験協力者

- ウェブ検索タスクにて意見文章の提出まで行い，事後アンケートへの回答が完了しているワーカー
- タスク所要時間が外れ値ではないワーカー

In [19]:
questionnaire_answered_crowdIds_csv_path = './data/questionnaire_answered_crowdId.csv'

questionnaire_answered_crowdIds = convert_csv_to_array(questionnaire_answered_crowdIds_csv_path)
questionnaire_answered_crowdIds_df = convert_singleArray_to_dataframe(
    questionnaire_answered_crowdIds, 'crowdId')

users_who_complete_webSearchTask_df = get_dataframe_from_db("select * from users where hasFinishedWebSearchTask")
users_who_completed_all_task_df =  users_who_complete_webSearchTask_df[users_who_complete_webSearchTask_df['crowdId'].isin(questionnaire_answered_crowdIds)]
users_who_completed_all_task_df.head()



<ipython-input-11-751477a6ed0c>:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.


,id,crowdId,password,taskId,hasFinishedWebSearchTask,created_at,updated_at
0,2,o1571139,"$argon2id$v=19$m=65536,t=3,p=4$Wriae75xWojEPuS...",VIETNAMTRIP_WITH_NONUI,1,2022-12-30 16:58:16,2022-12-30 17:22:07
1,3,takahiro0655,"$argon2id$v=19$m=65536,t=3,p=4$Brho2o4kb21R15K...",TPP_WITH_QUERYUI,1,2022-12-30 16:58:27,2022-12-30 17:24:54
2,4,bvglp11,"$argon2id$v=19$m=65536,t=3,p=4$aRJFbIAEEhWlQkg...",TPP_WITH_NONUI,1,2022-12-30 16:58:36,2022-12-30 17:13:53
3,5,a_11044,"$argon2id$v=19$m=65536,t=3,p=4$As4YyaA+x5HNXcf...",AIRPOLLUTION_WITH_QUESTIONUI,1,2022-12-30 16:58:51,2022-12-30 17:33:08
4,6,Hiroppi1,"$argon2id$v=19$m=65536,t=3,p=4$b+BKPMiKZvlrtL/...",AIRPOLLUTION_WITH_QUERYUI,1,2022-12-30 16:59:26,2022-12-30 17:08:34
